In [1]:

# Supervised Tuned
# https://cloud.google.com/vertex-ai/generative-ai/docs/models/gemini-supervised-tuning-about


#https://console.cloud.google.com/storage/browser/cloud-samples-data/ai-platform/generative_ai;tab=objects?prefix=&forceOnObjectsSortingFiltering=false


In [1]:
import json

In [7]:
def open_jsonl_file(filename):
  with open(filename, 'r') as f:
    data = [json.loads(line) for line in f]
  return data

In [18]:

data= open_jsonl_file("./gemini_pro_peft_train_sample.jsonl")

In [42]:
data[1]

{'messages': [{'role': 'user',
   'content': 'TRANSCRIPT: \nSUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.  She used to have allergies when she lived in Seattle but she thinks they are worse here.  In the past, she has tried Claritin, and Zyrtec.  Both worked for short time but then seemed to lose effectiveness.  She has used Allegra also.  She used that last summer and she began using it again two weeks ago.  It does not appear to be working very well.  She has used over-the-counter sprays but no prescription nasal sprays.  She does have asthma but doest not require daily medication for this and does not think it is flaring up.,MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: , She has no known medicine allergies.,OBJECTIVE:,Vitals:  Weight was 130 pounds and blood pressure 124/78.,HEENT:  Her throat was mildly erythematous without exudate.  Nasal mucosa was erythematous and swollen.  Only clear drainage wa

In [39]:
data[1]['messages'][0]

{'role': 'user',
 'content': 'TRANSCRIPT: \nSUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.  She used to have allergies when she lived in Seattle but she thinks they are worse here.  In the past, she has tried Claritin, and Zyrtec.  Both worked for short time but then seemed to lose effectiveness.  She has used Allegra also.  She used that last summer and she began using it again two weeks ago.  It does not appear to be working very well.  She has used over-the-counter sprays but no prescription nasal sprays.  She does have asthma but doest not require daily medication for this and does not think it is flaring up.,MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: , She has no known medicine allergies.,OBJECTIVE:,Vitals:  Weight was 130 pounds and blood pressure 124/78.,HEENT:  Her throat was mildly erythematous without exudate.  Nasal mucosa was erythematous and swollen.  Only clear drainage was seen.  TMs wer

In [33]:
data[1]['messages'][1]

{'role': 'model', 'content': 'Allergy / Immunology'}

In [35]:
classes= []
for d in data:
    if d['messages'][1].get("role") == 'model':
        classes.append(d['messages'][1].get("content"))

In [36]:
len(data)

519

In [37]:
set(classes)

{'Allergy / Immunology',
 'Autopsy',
 'Bariatrics',
 'Cardiovascular / Pulmonary',
 'Chiropractic',
 'Consult - History and Phy.',
 'Cosmetic / Plastic Surgery',
 'Dentistry',
 'Dermatology',
 'Diets and Nutritions',
 'Discharge Summary',
 'ENT - Otolaryngology',
 'Emergency Room Reports',
 'Endocrinology',
 'Gastroenterology',
 'General Medicine',
 'Hematology - Oncology',
 'Hospice - Palliative Care',
 'IME-QME-Work Comp etc.',
 'Lab Medicine - Pathology',
 'Letters',
 'Nephrology',
 'Neurology',
 'Neurosurgery',
 'Obstetrics / Gynecology',
 'Office Notes',
 'Ophthalmology',
 'Orthopedic',
 'Pain Management',
 'Pediatrics - Neonatal',
 'Physical Medicine - Rehab',
 'Podiatry',
 'Psychiatry / Psychology',
 'Radiology',
 'Rheumatology',
 'SOAP / Chart / Progress Notes',
 'Sleep Medicine',
 'Speech - Language',
 'Surgery',
 'Urology'}

In [11]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

In [12]:
from dotenv import dotenv_values
import json
from google.oauth2 import service_account
config = dotenv_values(".env")


with open('./complete-tube-421007-9a7c35cd44e2.json') as source:
    info = json.load(source)

vertex_credentials = service_account.Credentials.from_service_account_info(info)
vertexai.init(project=config['PROJECT'], location=config['REGION'],  credentials=vertex_credentials)

In [43]:
def multiturn_generate_content(text):
    generation_config = {
    "max_output_tokens": 2048,
    "temperature": .1,
    "top_p": 1,
}
    safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}  
    model = GenerativeModel(
    "projects/124210009832/locations/europe-west2/endpoints/782504833302134784",
    system_instruction=["""You are a model able to classify a text"""]
  )
    chat = model.start_chat()
    response = chat.send_message(
      [text],
      generation_config=generation_config,
      safety_settings=safety_settings
  )

    return response



In [44]:
text1_1 = """
classify the text using these classes in the list ['Allergy / Immunology',  'Autopsy', 'Bariatrics', 'Cardiovascular / Pulmonary', 'Chiropractic',
 'Consult - History and Phy.', 'Cosmetic / Plastic Surgery', 'Dentistry', 'Dermatology', 'Diets and Nutritions',
 'Discharge Summary', 'ENT - Otolaryngology', 'Emergency Room Reports', 'Endocrinology', 'Gastroenterology',
 'General Medicine', 'Hematology - Oncology', 'Hospice - Palliative Care', 'IME-QME-Work Comp etc.', 'Lab Medicine - Pathology',
 'Letters', 'Nephrology', 'Neurology', 'Neurosurgery', 'Obstetrics / Gynecology', 'Office Notes', 'Ophthalmology', 'Orthopedic',
 'Pain Management', 'Pediatrics - Neonatal', 'Physical Medicine - Rehab', 'Podiatry', 'Psychiatry / Psychology', 'Radiology',
 'Rheumatology', 'SOAP / Chart / Progress Notes', 'Sleep Medicine', 'Speech - Language', 'Surgery', 'Urology']

Text # 'TRANSCRIPT: \\nREVIEW OF SYSTEMS,GENERAL/CONSTITUTIONAL: , The patient denies fever, fatigue, weakness, weight gain or weight loss.,HEAD, EYES, EARS, NOSE AND THROAT:,  Eyes - The patient denies pain, redness, loss of vision, double or blurred vision, flashing lights or spots, dryness, the feeling that something is in the eye and denies wearing glasses.  Ears, nose, mouth and throat.  The patient denies ringing in the ears, loss of hearing, nosebleeds, loss of sense of smell, dry sinuses, sinusitis, post nasal drip, sore tongue, bleeding gums, sores in the mouth, loss of sense of taste, dry mouth, dentures or removable dental work, frequent sore throats, hoarseness or constant feeling of a need to clear the throat when nothing is there, waking up with acid or bitter fluid in the mouth or throat, food sticking in throat when swallows or painful swallowing.,CARDIOVASCULAR: , The patient denies chest pain, irregular heartbeats, sudden changes in heartbeat or palpitation, shortness of breath, difficulty breathing at night, swollen legs or feet, heart murmurs, high blood pressure, cramps in his legs with walking, pain in his feet or toes at night or varicose veins.,RESPIRATORY: , The patient denies chronic dry cough, coughing up blood, coughing  up mucus, waking at night coughing or choking, repeated pneumonias, wheezing or night sweats.,GASTROINTESTINAL: , The patient denies decreased appetite, nausea, vomiting, vomiting blood or coffee ground material, heartburn, regurgitation, frequent belching, stomach pain relieved by food, yellow jaundice, diarrhea, constipation, gas, blood in the stools, black tarry stools or hemorrhoids.,GENITOURINARY:  ,The patient denies difficult urination, pain or burning with urination, blood in the urine, cloudy or smoky urine, frequent need to urinate, urgency, needing to urinate frequently at night, inability to hold the urine, discharge from the penis, kidney stones, rash or ulcers, sexual difficulties, impotence or prostate trouble, no sexually transmitted diseases.,MUSCULOSKELETAL: , The patient denies arm, buttock, thigh or calf cramps.  No joint or muscle pain.  No muscle weakness or tenderness.  No joint swelling, neck pain, back pain or major orthopedic injuries.,SKIN AND BREASTS:  ,The patient denies easy bruising, skin redness, skin rash,  hives, sensitivity to sun exposure, tightness, nodules or bumps, hair loss, color changes in the hands or feet with cold, breast lump, breast pain or nipple discharge.,NEUROLOGIC: , The patient denies headache, dizziness, fainting, muscle spasm, loss of consciousness, sensitivity or pain in the hands and feet or memory loss.,PSYCHIATRIC:  ,The patient denies depression with thoughts of suicide, voices in ?? head telling ?? to do things and has not been seen for psychiatric counseling or treatment.,ENDOCRINE: , The patient denies intolerance to hot or cold temperature, flushing, fingernail changes, increased thirst, increased salt intake or decreased sexual desire.,HEMATOLOGIC/LYMPHATIC:  ,The patient denies anemia, bleeding tendency or clotting tendency.,ALLERGIC/IMMUNOLOGIC: , The patient denies rhinitis, asthma, skin sensitivity, latex allergies or sensitivity.\\n\\n 
LABEL:' """



multiturn_generate_content(text1_1)

candidates {
  content {
    role: "model"
    parts {
      text: "Consult - History and Phy."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.172589809
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0701762959
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.298491359
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0601970479
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.22303842
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.124958552
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.114368536
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0626743659
  }
}
usage_metadata {
  prompt_token_count: 948
  candidates_token_count: 

In [ ]:
# Example Fine Tune with code


import vertexai
from vertexai.generative_models import GenerativeModel
from vertexai.preview.tuning import sft

vertexai.init(project="complete-tube-421007", location="europe-west2")

gemini_pro = GenerativeModel("gemini-1.0-pro-002")

sft_tuning_job = sft.train(
    source_model=gemini_pro,
    train_dataset="gs://cloud-samples-data/vertex-ai/model-evaluation/gemini_pro_peft_train_sample.jsonl",
    validation_dataset="gs://cloud-samples-data/vertex-ai/model-evaluation/gemini_pro_peft_eval_sample.jsonl",
    tuned_model_display_name="test2",
    epochs=4,
    learning_rate_multiplier=1,
)
